In [99]:
import numpy as np
import pandas as pd
import matplotlib as plt
from torch import torch, nn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [100]:
torch.cuda.is_available()

True

In [101]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1080'

In [102]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Load the heart disease dataset:
columns = ['Age', 'Oldpeak', 'MaxHR', 'HeartDisease']
heart_disease_dataframe = pd.read_csv("../../Data/heart.csv")[columns]

# Select out X and Y
X = heart_disease_dataframe[['Age', 'Oldpeak', 'MaxHR']].values
y = heart_disease_dataframe['HeartDisease'].values

# Apply standard scaler to X.
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

X_train_tensor = torch.Tensor(torch.from_numpy(X_train.astype(np.float32)))
y_train_tensor = torch.Tensor(torch.from_numpy(y_train.astype(np.float32))).reshape(-1,1)
X_test_tensor = torch.Tensor(torch.from_numpy(X_test.astype(np.float32)))
y_test_tensor = torch.Tensor(torch.from_numpy(y_test.astype(np.float32))).reshape(-1,1)

In [114]:
class NN(torch.nn.Module):
    def __init__(self, input_dimensions, num_hidden_units=10):
        super(NN, self).__init__()
        self.linear = torch.nn.Linear(input_dimensions, num_hidden_units)
        self.relu1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(num_hidden_units, 1)
        self.output = torch.nn.Sigmoid()

    def forward(self, x):
        z1 = self.linear(x)
        a1 = self.relu1(z1)
        z2 = self.linear2(a1)
        return self.output(z2)


model = NN()
loss_function = torch.nn.BCELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)
errors = []
for epoch in range(200):  
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = loss_function(outputs, y_train_tensor)
    errors.append(loss.item())
    loss.backward(retain_graph=True)
    #print(model.linear.weight.data, model.linear.bias.data)
    optimizer.step()
plt.plot(errors)

TypeError: 'int' object is not subscriptable